In [1]:
# 准备工作

import warnings

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 加载数据
train_tf = pd.read_csv("./datasets/train.csv")
test_tf = pd.read_csv("./datasets/test.csv")
train_data = train_tf.drop(["label"], axis=1)
train_label = train_tf["label"]

In [3]:
train_label = pd.get_dummies(train_label)

In [4]:
X_train, X_test, y_train,  y_test = train_test_split(train_data, 
                                                     train_label, test_size=0.2, 
                                                     random_state=42, stratify=train_label)


In [5]:
m, n = X_train.shape
X_train.shape

(33600, 784)

In [6]:
# 神经网络参数
learning_rate = 0.001
num_steps = 500
batch_size = 50
display_step = 10

num_input = 784
num_classes = 10
dropout = 0.75

# tf 计算图输入
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

In [7]:
# conv2d的封装层
def conv2d(x, W, b, strides=1):
    # strides 步长为1
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding="SAME")
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [8]:
# 池化层的封装
def maxpool2d(x, k=2):  
    # k 池化大小
    result = tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], 
                           padding="SAME")
    return result
    

In [9]:
# cnn模型结构
def conv_net(x, weights, biases, dropout):
    # input: [batch_size, height, weight, channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # 卷积层
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, 2)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, 2)
    
    # 全连接层
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [10]:
# 权重和偏置
weights = {
     # 5*5 conc, 1 input, 32 outputs
    'wc1':tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2':tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1':tf.Variable(tf.random_normal([7 * 7 * 64, 1024])), 
    'out':tf.Variable(tf.random_normal([1024, num_classes]))
}
biases = {
    'bc1':tf.Variable(tf.random_normal([32])),
    'bc2':tf.Variable(tf.random_normal([64])),
    'bd1':tf.Variable(tf.random_normal([1024])),
    'out':tf.Variable(tf.random_normal([num_classes]))
}

In [11]:
# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     sess.run(init)
#     a = weights['wd1'].get_shape()
#     print(type(a))

In [12]:
# 输出结果
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

In [13]:
# 定义损失和优化器
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [14]:
# 评估模型
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1)) # true or false
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [15]:
# 变量初始化
init = tf.global_variables_initializer()

In [21]:
# 开始训练
with tf.Session() as sess:
    sess.run(init)
    for step in range(1):
        index = np.random.randint(m - batch_size)
        batch_x = X_train[index: index + batch_size].values
        batch_y = y_train[index: index + batch_size].values

#         sess.run(train_op, feed_dict={X:batch_x, Y:batch_y, keep_prob: keep_prob})
#         if step % display_step == 0:
#             # 计算batch 的损失和精确度
#             loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y:batch_y})
#             print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(loss) + 
#                   ", Training Accuracy= " + "{:.3f}".format(acc) )
    print("Optimization Finished!")

<class 'numpy.ndarray'>
Optimization Finished!
